In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.stats as ss
import seaborn as sns
sc.settings.set_figure_params(dpi=100)

### Read in Data

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc, roc_curve, precision_recall_curve, f1_score, balanced_accuracy_score, accuracy_score
import pickle as pkl
# read in data
a_trb = sc.read_h5ad('../outs/adata.trb.h5ad')
trbs_covid = pd.read_csv('../outs/su22.trb.csv', index_col=0)
# load the pickled data
with open('../external_data/results.tcr.pkl', 'rb') as f:
    results_tcr = pkl.load(f)
with open('../external_data/results.ag.pkl', 'rb') as f:
    results_ag = pkl.load(f)

#### Fetal Donors (Suo and Dann et al. 2022)

In [ ]:
from tqdm import tqdm
# define the minimum number of cells
min_cells = 4
# get the tag, keeping only pairs that have at least min_cells cells
clusters = ['CD8+T','TYPE_1_INNATE_T']
mask = results_tcr['SUO_SCIENCE2022_FETAL']['celltype_annotation'].isin(clusters)
data = results_tcr['SUO_SCIENCE2022_FETAL'].loc[mask, ['donor','celltype_annotation','TRB']].astype(str).copy()
data['tag'] = data[['donor','celltype_annotation']].astype(str).agg(':'.join, axis=1)
# filter the data more harshly because less assured of quality
data['TRB'][~data['TRB'].isin(a_trb.obs.index)] = np.nan
data = data.dropna(subset=['TRB'])
counts = data['tag'].value_counts(); tags = counts.index[counts >= min_cells]
# compile the Xs
Xs = []
for tag in tqdm(tags):
    trbs = data.loc[data['tag'] == tag, 'TRB']
    mask = trbs[trbs.isin(a_trb.obs.index)]
    X_ = pd.Series(a_trb[mask].X.mean(0), name=tag)
    Xs.append(X_)
og_trb_suo2022_X = pd.concat(Xs, axis=1).T

#### Adults with COVID-19 and Healthy Donors (Su, Yuan, and Chen et al. 2022)

In [ ]:
# derive annotations
adata = sc.read_h5ad('../../COVID_ISB_STORAGE/upto_v16_P_GE_int_gex_cd8_t_cells.has_abtcr_chain.h5ad')
results_tcr['SU_CELL2022_COVID19']['phenotype_leiden'] = adata.obs[['phenotype','leiden']].agg('_'.join, axis=1)
results_tcr['SU_CELL2022_COVID19'][['batch','subbatch','sample']] = \
results_tcr['SU_CELL2022_COVID19']['batch_info'].str.split(':', expand=True)
# compile data
trbs_atlas = pd.DataFrame(a_trb.X, index=a_trb.obs.index)
trbs_atlas = trbs_atlas.loc[~trbs_atlas.index.isin(trbs_covid.index)]
trbs_covid.columns = trbs_atlas.columns
trbs_X = pd.concat([trbs_atlas, trbs_covid], axis=0)
assert trbs_X.index.is_unique
# derive annotations
results_tcr['SU_CELL2022_COVID19'][['batch','subbatch','sample']] = \
results_tcr['SU_CELL2022_COVID19']['batch_info'].str.split(':', expand=True)
# get the tag, keeping only pairs that have at least min_cells cells
mask = ~results_tcr['SU_CELL2022_COVID19']['phenotype_leiden'].isna()
data = results_tcr['SU_CELL2022_COVID19'][['sample','phenotype_leiden','TRB']].astype(str).copy()
data['tag'] = data[['sample','phenotype_leiden']].astype(str).agg(':'.join, axis=1)
# filter the data more harshly because less assured of quality
data['TRB'][~data['TRB'].isin(a_trb.obs.index)] = np.nan
data = data.dropna(subset=['TRB'])
counts = data['tag'].value_counts(); tags = counts.index[counts >= min_cells]
# compile the Xs
Xs = []
for tag in tqdm(tags):
    trbs = data.loc[data['tag'] == tag, 'TRB']
    mask = trbs[trbs.isin(a_trb.obs.index)]
    X_ = pd.Series(a_trb[mask].X.mean(0), name=tag)
    Xs.append(X_)
og_trb_su2022_X = pd.concat(Xs, axis=1).T

#### Pan-Cancer Types (Zheng, Qin, and Si et al. 2021)

In [ ]:
# get the tag, keeping only pairs that have at least min_cells cells
mask = results_tcr['ZHENG_SCIENCE2021_PANCAN']['TcellType'] == 'CD8'
data = results_tcr['ZHENG_SCIENCE2021_PANCAN'].loc[mask, ['patient','cluster.name','TRB']].astype(str).copy()
data['tag'] = data[['patient','cluster.name']].astype(str).agg(':'.join, axis=1)
# filter the data more harshly because less assured of quality
data['TRB'][~data['TRB'].isin(a_trb.obs.index)] = np.nan
data = data.dropna(subset=['TRB'])
counts = data['tag'].value_counts(); tags = counts.index[counts >= min_cells]
# compile the Xs
Xs = []
for tag in tqdm(tags):
    trbs = data.loc[data['tag'] == tag, 'TRB']
    mask = trbs[trbs.isin(a_trb.obs.index)]
    X_ = pd.Series(a_trb[mask].X.mean(0), name=tag)
    Xs.append(X_)
og_trb_zheng2021_X = pd.concat(Xs, axis=1).T

### Modeling

In [ ]:
from sklearn.linear_model import LogisticRegression
# define a function to interrogate the data
def interrogate_with_globals():
    # create statistics tracking dataframe
    df_stat = pd.DataFrame(columns=['auroc','auprc','f1_score','balacc'])
    # create tracking variables for downstream visualization and statistics
    probas, probas_bin, truths = [], [], []
    fprs, tprs, pres, recs = [], [], [], []
    # train utilizing random forest models in a stratified shuffled manner
    skf = StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=1/4)
    for idxs_train, idxs_test in skf.split(X1, y1):
        # instantiate the random forest model
        clf = LogisticRegression()
        # fit the random forest model using Dataset #1
        clf = clf.fit(X1.iloc[idxs_train], y1.iloc[idxs_train])

        # predict on Dataset #2 correcting to all indices if requested
        if pred_on_all:
            idxs_test = range(X2.shape[0])
        # derive the probabilities
        proba = clf.predict_proba(X2.iloc[idxs_test])[:, clf.classes_ == 1]
        probas.append(pd.Series(proba[:, 0], index=X2.index[idxs_test]))
        # binarize into categorical predictions
        proba_bin = 1 * (proba >= 0.50)
        probas_bin.append(pd.Series(proba_bin[:, 0], index=X2.index[idxs_test]))
        # retrieve the associated ground truth
        truth = y2.iloc[idxs_test]
        truths.append(truth.copy())

        # compute subsequent AUROC and AUPRC related metrics
        fpr, tpr, _ = roc_curve(truth, proba)
        pre, rec, _ = precision_recall_curve(truth, proba)
        fprs.append(fpr); tprs.append(tpr); pres.append(pre); recs.append(rec)
        # save the relevant statistics
        df_stat.loc[df_stat.shape[0]] = auc(fpr, tpr), auc(rec, pre), \
                                        f1_score(truth, proba_bin, average='binary'), \
                                        balanced_accuracy_score(truth, proba_bin)

    # check the difference
    for stat in df_stat.columns:
        fig, ax = plt.subplots(figsize=[1, 4]); ax.grid(False)
        sns.boxplot(y=df_stat[stat], linewidth=1.5, saturation=1, showfliers=False, linecolor='dodgerblue', color='skyblue')
        sns.stripplot(y=df_stat[stat], linewidth=1.5, s=6, alpha=0.5, color='skyblue', edgecolor='dodgerblue')
        ax.set_xlim(-0.75, 0.75); ax.set_ylabel(stat.upper())
        print(stat.upper(), df_stat[stat].mean(), df_stat[stat].std() / np.sqrt(df_stat.shape[0])*1.96)

    # plot the FPR, TPR
    fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
    xl = np.arange(0, 1.01, 0.01); yls = []
    for fpr, tpr in zip(fprs, tprs):
        ax.plot(fpr, tpr, color='skyblue', linestyle='--', lw=1)
        yls.append(np.interp(xl, fpr, tpr))
    yl = np.vstack(yls).mean(0)
    ax.plot(xl, yl, color='dodgerblue', lw=2)
    ax.plot([0, 1], [0, 1], color='lightgray', linestyle='dotted')
    ax.set(xlabel='False Positive Rate', ylabel='True Positive Rate')

    # plot the precision recall
    fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
    xl = np.arange(0, 1.01, 0.01); yls = []
    for pre, rec in zip(pres, recs):
        ax.plot(rec[::-1], pre[::-1], color='skyblue', linestyle='--', lw=1)
        yls.append(np.interp(xl, rec[::-1], pre[::-1]))
    yl = np.vstack(yls).mean(0)
    ax.plot(xl, yl, color='dodgerblue', lw=2)
    ax.plot([0, 1], [0.5]*2, color='lightgray', linestyle='dotted')
    ax.set(xlabel='Recall', ylabel='Precision')

    from sklearn.metrics import confusion_matrix
    # define the results
    result = pd.DataFrame(columns=['Truth','PredProb'])
    for proba, truth in zip(probas, truths):
        result.loc[result.shape[0]] = '+', proba[truth == 1].mean()
        result.loc[result.shape[0]] = '-', proba[truth == 0].mean()

    # compare the average prediction probabilities
    fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
    sns.boxplot(x='Truth', y='PredProb', data=result, linewidth=1.5, saturation=1,
                showfliers=False, linecolor='dodgerblue', color='skyblue',
                order=['-', '+'], palette=['lightgray','skyblue'])
    np.random.seed(0)
    sns.stripplot(x='Truth', y='PredProb', data=result, jitter=0.4, palette=['dodgerblue'], order=['+'], alpha=0.6, s=6)
    sns.stripplot(x='Truth', y='PredProb', data=result, jitter=0.4, palette=['grey'], order=['-'], alpha=0.6, s=6)
    ax.set_xlim(-1, 2); ax.set_ylabel('Prediction Probability'); ax.set_xlabel('Ground Truth')
    ax.get_children()[0].set_hatch('//')
    ax.get_children()[0].set_edgecolor('grey')
    for idx in range(1, 6):
        ax.get_children()[idx].set_color('grey')

    # report statistics
    print('p-value for + vs. -:')
    print(ss.mannwhitneyu(result.loc[result['Truth'] == '+', 'PredProb'], result.loc[result['Truth'] == '-', 'PredProb']))
    print('average:')
    print(df_stat.mean(0))
    print('95% cis:')
    print(df_stat.std(0) / np.sqrt(10) * 1.96)
    return df_stat, probas, truths

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#### Fetal Donors...

In [ ]:
# define the data to train on
X1 = og_trb_suo2022_X.copy()
# setup a mask for CD8+ cells
y1 = pd.Series(X1.index.str.contains(':TYPE_1_INNATE_T'), index=X1.index)
print(X1.shape[0], y1.sum(), y1.mean())

# define the data to predict on
X2 = og_trb_suo2022_X.copy()
# setup a mask for CD8+ cells
y2 = pd.Series(X2.index.str.contains(':TYPE_1_INNATE_T'), index=X2.index)
print(X2.shape[0], y2.sum(), y2.mean())

# define whether we are to predict on the complete data
pred_on_all = False
# perform predictions with all
df_stat_fetal, _, _ = interrogate_with_globals()

#### Fetal... --> COVID-19 and Adult Healthy Donors...

In [ ]:
# define the data to train on
X1 = og_trb_suo2022_X.copy()
# setup a mask for CD8+ cells
y1 = pd.Series(X1.index.str.contains(':TYPE_1_INNATE_T'), index=X1.index)
print(X1.shape[0], y1.sum(), y1.mean())

# define the data to predict on
X2 = og_trb_su2022_X.copy()
X2 = X2.loc[~X2.index.str.endswith('nan')]
X2 = X2.loc[~X2.index.str.endswith('CD8_Naive_14')]
# setup a mask for CD8+ cells
y2 = pd.Series(X2.index.str.contains('CD8_Cytotoxic_3'), index=X2.index)
print(X2.shape[0], y2.sum(), y2.mean())

# define whether we are to predict on the complete data
pred_on_all = True
# perform predictions with all
df_stat_covid, probas_covid, _ = interrogate_with_globals()

In [ ]:
# visualize the distribution
df_plot = pd.concat(probas_covid, axis=1).reset_index().melt(id_vars='index')
df_plot[['sample','phenotype']] = df_plot['index'].str.split(':', expand=True)
df_plot['value'] -= df_plot['value'].mean()
df_plot['value'] /= df_plot['value'].std()
fig, ax = plt.subplots(figsize=[5, 2.5]); ax.grid(False)
order = df_plot.groupby('phenotype').mean(numeric_only=True)['value'].sort_values().index
sns.barplot(x='phenotype', y='value', data=df_plot, ci=95, errwidth=1.5, linewidth=1.5, ax=ax, order=order,
            capsize=0.3, errcolor='dodgerblue', edgecolor='dodgerblue', color='skyblue', saturation=1)
ax.tick_params(axis='x', labelrotation=90)
ax.axhline(0, color='k')
ax.set_xlim(-1, len(order))
ax.set(xlabel='Adult PT and HD CD8+ T Cell States', ylabel='Type I Innate Score')

#### Fetal... --> Pan-Cancer...

In [ ]:
# define the data to train on
X1 = og_trb_suo2022_X.copy()
# setup a mask for CD8+ cells
y1 = pd.Series(X1.index.str.contains(':TYPE_1_INNATE_T'), index=X1.index)
print(X1.shape[0], y1.sum(), y1.mean())

# define the data to predict on
X2 = og_trb_zheng2021_X.copy()
# setup a mask for CD8+ cells
y2 = pd.Series(X2.index.str.contains('KIR'), index=X2.index)
print(X2.shape[0], y2.sum(), y2.mean())

# define whether we are to predict on the complete data
pred_on_all = True
# perform predictions with all
df_stat_tumor, probas_tumor, _ = interrogate_with_globals()

In [ ]:
# visualize the distribution
df_plot = pd.concat(probas_tumor, axis=1).reset_index().melt(id_vars='index')
df_plot[['sample','phenotype']] = df_plot['index'].str.split(':', expand=True)
df_plot['value'] -= df_plot['value'].mean()
df_plot['value'] /= df_plot['value'].std()
fig, ax = plt.subplots(figsize=[5, 4]); ax.grid(False)
order = df_plot.groupby('phenotype').mean(numeric_only=True)['value'].sort_values().index
sns.barplot(x='phenotype', y='value', data=df_plot, ci=95, errwidth=1.5, linewidth=1.5, ax=ax, order=order,
            capsize=0.3, errcolor='dodgerblue', edgecolor='dodgerblue', color='skyblue', saturation=1)
ax.tick_params(axis='x', labelrotation=90)
ax.axhline(0, color='k')
ax.set_xlim(-1, len(order))
ax.set(xlabel='Pan-Cancer CD8+ T Cell States', ylabel='Type I Innate Score')

### Confirm Predictions with Transcriptomic Analyses

In [ ]:
# read in the original H5AD file
adata = sc.read_h5ad('../external_data/SUO_SCIENCE2022_FETAL/PAN.A01.v01.raw_count.20210429.NKT.embedding.abTCR.h5ad')
# normalize the data
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)
# can we isolate out the two populations?
adata_ = adata[adata.obs['celltype_annotation'] == 'TYPE_1_INNATE_T'].copy()
sc.pp.neighbors(adata_, use_rep='X_scvi', n_neighbors=30)
sc.tl.umap(adata_)
# compute the leidens
sc.tl.leiden(adata_, resolution=0.9)
adata_.obs['phenotype'] = np.nan
adata_.obs['phenotype'][adata_.obs['leiden'].isin(['3','4','7'])] = 'MAIT-like'
adata_.obs['phenotype'][adata_.obs['leiden'].isin(['0','1','5'])] = 'KIR-like'
# write out the data
adata_.write('../outs/fetal.toit.h5ad')

# color on the phenotype
adata_ = sc.read_h5ad('../outs/fetal.toit.h5ad')
adata_.obs['phenotype'] = adata_.obs['phenotype'].astype(str)
adata_.obs['phenotype'] = adata_.obs['phenotype'].str.replace('nan','Interm.')
adata_.uns['phenotype_colors'] = [x for x in sns.color_palette('Set2', 3).as_hex()]
sc.pl.umap(adata_, color=['phenotype'])

In [ ]:
from matplotlib.colors import to_rgb, to_hex
# change colors
colors = [list(to_rgb(x)) for x in adata_.uns['phenotype_colors']]
colors_neo = []
for color in colors:
    color[-1] *= 2.00
    color[0] *= 1.25
    color[0] = min(1, color[0])
    color[-1] = min(1, color[-1])
    color = np.array(color) * 0.95
    colors_neo.append(to_hex(color))
adata_.uns['phenotype_colors'] = colors_neo
sc.pl.umap(adata_, color=['phenotype'])

#### KIR and MAIT -like Signatures

In [ ]:
# look for marker genes
sc.pl.dotplot(adata_, groupby='phenotype', use_raw=False, dot_max=0.80, vmin=0, vmax=8,
              categories_order=['MAIT-like','Interm.','KIR-like'],
              var_names=['NKG7','KLRB1','IL7R','CXCR4','IFNGR1','IL23R','CXCR6','RORC',
                         'ZBTB16','GATA3','IFNG','ZNF683','ICOS','TCF7','LEF1','KLRD1','IL10RA',
                         'TOX','PDCD1','TNFRSF9','KIR3DL1','KIR3DL2','KIR2DL1','KIR2DL3',], cmap='Blues')

In [ ]:
# compute the MAGIC imputed gene expression
mdata_ = adata_.copy()
sc.pp.filter_genes(mdata_, min_cells=1)
sc.external.pp.magic(mdata_, random_state=0, t=3)
# define the gene lists
genes_mait = ['RORC','KLRB1','CXCR6','IL23R']
genes_kir = ['KIR2DL1','KIR2DL3','KIR3DL1','KIR3DL2']
# derive the signatures
sc.tl.score_genes(mdata_, gene_list=genes_mait, score_name='score_MAIT', random_state=0, use_raw=False)
sc.tl.score_genes(mdata_, gene_list=genes_kir, score_name='score_KIR', random_state=0, use_raw=False)
# plot the signatures
mdata_.obs[['score_MAIT','score_KIR']] -= mdata_.obs[['score_MAIT','score_KIR']].mean()
mdata_.obs[['score_MAIT','score_KIR']] /= mdata_.obs[['score_MAIT','score_KIR']].std()
# look at the scores but weighted accordingly
for k in ['score_MAIT','score_KIR']:
    vs = mdata_.obs[k].copy()
    vmin, vmax = np.percentile(vs, 2), np.percentile(vs, 98)
    sc.pl.umap(mdata_, color=[k], cmap='Blues', vmin=vmin, vmax=vmax)

#### TCR Gene Usage

In [ ]:
# get the counts for each category
genes = adata_.obs.loc[adata_.obs['leiden'].isin(['3','4','7']), 'IR_VJ_1_v_gene'].dropna()
counts_mait = genes[~genes.str.lower().isin(['nan','none'])].astype(str).value_counts()
genes = adata_.obs.loc[adata_.obs['leiden'].isin(['0','1','5']), 'IR_VJ_1_v_gene'].dropna()
counts_kir = genes[~genes.str.lower().isin(['nan','none'])].astype(str).value_counts()
genes = adata_.obs['IR_VJ_1_v_gene'].dropna()
counts_type1 = genes[~genes.str.lower().isin(['nan','none'])].astype(str).value_counts()
genes = adata.obs.loc[adata.obs['celltype_annotation'].isin(['CD8+T']), 'IR_VJ_1_v_gene'].dropna()
counts_cd8 = genes[~genes.str.lower().isin(['nan','none'])].astype(str).value_counts()

In [ ]:
# derive the order for TRAV
idxs = counts_kir.index.union(counts_mait.index).union(counts_type1.index).union(counts_cd8.index)
converts = idxs.to_series().str.slice(4).str.split('/DV', expand=True)[0].str.split('-', expand=True).astype(float)
converts[1] = converts[1].fillna(0) / 10; converts = converts.sum(1)
order = idxs.sort_values(key=lambda x: converts.loc[x])
# look at the different TRAV usages
x, y = counts_kir.reindex(idxs).fillna(0) / counts_kir.sum() * 100, counts_mait.reindex(idxs).fillna(0) / counts_mait.sum() * 100
z = counts_type1.reindex(idxs).fillna(0) / counts_type1.sum() * 100
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
ax.scatter(x - z, y - z, s=z*10, color='skyblue')
ax.scatter(x.loc['TRAV1-2'] - z.loc['TRAV1-2'], y.loc['TRAV1-2'] - z.loc['TRAV1-2'], color='dodgerblue',
           edgecolor='k', s=z.loc['TRAV1-2']*10)
ax.set(xlabel='TRAV % KIR-like - Type I', ylabel='TRAV % MAIT-like - Type I')
ax.axhline(0, color='k', linestyle='--'); ax.axvline(0, color='k', linestyle='--')
# plot the levels over genomic distance
fig, ax = plt.subplots(figsize=[12, 4]); ax.grid(False)
ax.scatter(order, y.loc[order] - x.loc[order], color='skyblue', s=z*10)
ax.scatter(['TRAV1-2'], y.loc['TRAV1-2'] - x.loc['TRAV1-2'], color='dodgerblue', s=z.loc['TRAV1-2']*10, edgecolor='k')
ax.tick_params(axis='x', labelrotation=90); ax.axhline(0, color='k', linestyle='--')
ax.set_ylabel('TRAV % MAIT-like - KIR-like')
# derive the correlation
mask = ~x.isna(); ss.pearsonr(x[mask] - z[mask], y[mask] - z[mask])

In [ ]:
# get the counts for each category
genes = adata_.obs.loc[adata_.obs['leiden'].isin(['3','4','7']), 'IR_VJ_1_j_gene'].dropna()
counts_mait = genes[~genes.str.lower().isin(['nan','none'])].astype(str).value_counts()
genes = adata_.obs.loc[adata_.obs['leiden'].isin(['0','1','5']), 'IR_VJ_1_j_gene'].dropna()
counts_kir = genes[~genes.str.lower().isin(['nan','none'])].astype(str).value_counts()
genes = adata_.obs['IR_VJ_1_j_gene'].dropna()
counts_type1 = genes[~genes.str.lower().isin(['nan','none'])].astype(str).value_counts()
genes = adata.obs.loc[adata.obs['celltype_annotation'].isin(['CD8+T']), 'IR_VJ_1_j_gene'].dropna()
counts_cd8 = genes[~genes.str.lower().isin(['nan','none'])].astype(str).value_counts()

In [ ]:
# derive the order for TRAV
idxs = counts_kir.index.union(counts_mait.index).union(counts_type1.index).union(counts_cd8.index)
converts = idxs.to_series().str.slice(4).str.split('/DV', expand=True)[0].str.split('-', expand=True).astype(float)[0]
order = idxs.sort_values(key=lambda x: converts.loc[x])
# look at the different TRAV usages
x, y = counts_kir.reindex(idxs).fillna(0) / counts_kir.sum() * 100, counts_mait.reindex(idxs).fillna(0) / counts_mait.sum() * 100
z = counts_type1.reindex(idxs).fillna(0) / counts_type1.sum() * 100
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
ax.scatter(x - z, y - z, s=z*10, color='skyblue')
for gene in ['TRAJ33','TRAJ20','TRAJ12']:
    ax.scatter(x.loc[gene] - z.loc[gene], y.loc[gene] - z.loc[gene], color='dodgerblue', edgecolor='k', s=z.loc[gene]*10)
ax.set(xlabel='TRAJ % KIR-like - Type I', ylabel='TRAJ % MAIT-like - Type I')
ax.axhline(0, color='k', linestyle='--'); ax.axvline(0, color='k', linestyle='--')
# plot the levels over genomic distance
fig, ax = plt.subplots(figsize=[12, 4]); ax.grid(False)
ax.scatter(order, y.loc[order] - x.loc[order], color='skyblue', s=z*10)
for gene in ['TRAJ33','TRAJ20','TRAJ12']:
    ax.scatter([gene], y.loc[gene] - x.loc[gene], color='dodgerblue', s=z.loc[gene]*10, edgecolor='k')
ax.tick_params(axis='x', labelrotation=90); ax.axhline(0, color='k', linestyle='--')
ax.set_ylabel('TRAJ % MAIT-like - KIR-like')
# derive the correlation
mask = ~x.isna(); ss.pearsonr(x[mask] - z[mask], y[mask] - z[mask])

### Celltypist Mapping

In [ ]:
from scipy.sparse import csr_matrix
# read in the data
adata_ = sc.read_h5ad('../outs/fetal.toit.h5ad')
# reverse ln(CPM+1) by exp(X)-1
adata_.X = csr_matrix(np.exp(adata_.X.toarray())-1)
sc.pp.normalize_total(adata_, target_sum=1e4)
sc.pp.log1p(adata_)

import celltypist
# train the model
sc.pp.filter_genes(adata_, min_cells=1)
new_model = celltypist.train(adata_[~adata_.obs['phenotype'].isna()], labels='phenotype', n_jobs=10, feature_selection=True)
new_model.write('../outs/celltypist.toit.pkl')

In [ ]:
# now change the adata
adata_raw = sc.read_h5ad('../../COVID_ISB_STORAGE/upto_v16_P_GE_int_RAW_gex_cd8_t_cells.h5ad')
adata_covid.X = adata_raw[adata_covid.obs.index, adata_covid.var.index].X
# ^ verified as int
sc.pp.normalize_total(adata_covid, target_sum=1e4)
sc.pp.log1p(adata_covid)

# inference via the model
predictions = celltypist.annotate(adata_covid, model='../outs/celltypist.toit.pkl', majority_voting=False)
adata_covid.obs[predictions.probability_matrix.columns] = predictions.probability_matrix
adata_covid.obs['conf_score'] = predictions.to_adata().obs['conf_score']
# examine MAIT-like cell placement
fig, ax = plt.subplots(figsize=[6, 4]); ax.grid(False)
sns.barplot(x='phenotype_leiden', y='MAIT-like_TYPE_1_INNATE_T', data=adata_covid.obs,
            order=adata_covid.obs['phenotype_leiden'].cat.categories,
            palette=adata_covid.uns['phenotype_leiden_colors'], ci=95, capsize=0.3,
            errcolor='#2b2b2b', edgecolor='#2b2b2b', linewidth=1.5, errwidth=1.5)
ax.tick_params(axis='x', labelrotation=90)